### Описание проекта
Интернет-магазину необходимо автоматически выделять токсичные/негативные комментарии и отправлять их на модерацию. 
Для этого нужно обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Метрика качества: *F1* не меньше 0.75. 

### План проекта

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

Ознакомимся с дататсетом

In [1]:
import pandas as pd

comments = pd.read_csv('//datasets/toxic_comments.csv')
print(comments)

                                                     text  toxic
0       Explanation\nWhy the edits made under my usern...      0
1       D'aww! He matches this background colour I'm s...      0
2       Hey man, I'm really not trying to edit war. It...      0
3       "\nMore\nI can't make any real suggestions on ...      0
4       You, sir, are my hero. Any chance you remember...      0
...                                                   ...    ...
159566  ":::::And for the second time of asking, when ...      0
159567  You should be ashamed of yourself \n\nThat is ...      0
159568  Spitzer \n\nUmm, theres no actual article for ...      0
159569  And it looks like it was actually you who put ...      0
159570  "\nAnd ... I really don't think you understand...      0

[159571 rows x 2 columns]


In [2]:
print(comments['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64


Мы видим, что целевой признак содержится в столбце toxic, комментарии в столбце text. Создадим корпус текстов из каждого комментария.

In [3]:
corpus = list(comments['text'])

Напишем формулу для лемматизации текста

In [4]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import re

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    s = nltk.word_tokenize(text)
    string = ' '.join([lemmatizer.lemmatize(w) for w in s])
    return string

Напишем формулу для очистки текста от регулярных выражений

In [6]:
def clear_text(text):
    text_cleared = re.sub(r'[^a-zA-Z\']', ' ', text)
    text_cleared = " ".join(text_cleared.split())
    return text_cleared

Обработаем корпус при помощи данных формул

In [7]:
%%time
for i in range(len(comments['text'])):
    #print(i)
    corpus[i] = lemmatize(clear_text(corpus[i]))

CPU times: user 2min 10s, sys: 569 ms, total: 2min 10s
Wall time: 2min 11s


In [8]:
#для примера выведем первый текст корпуса
print(corpus[0])

Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They were n't vandalism just closure on some GAs after I voted at New York Dolls FAC And please do n't remove the template from the talk page since I 'm retired now


Выделим целевой признак

In [9]:
target = comments['toxic']
print(target)

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 159571, dtype: int64


Разобьем данные на обучающую и тестовую выборки. На тест отведем 25% генеральной совокупности.

In [10]:
from sklearn.model_selection import train_test_split
corpus_train, corpus_test, target_train, target_test = train_test_split(corpus, target, test_size=0.25, random_state=12345)

Для очистки текста от не несущих эмоциональную окраску местоимений и предлогов загрузим стопслова

In [11]:
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Применим метод TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)

In [13]:
tf_idf_test = count_tf_idf.transform(corpus_test)

### Вывод
Мы лемматизировали текст, очистили от ненужных символов и стопслов. Векторизовали корпус текстов, разделили на обучающую и тестовую выборку

# 2. Обучение

Проанализируем логистическую регрессию, решающее дерево, случайный лес.
Начнем с обучения логистической регрессии. Качество модели будем оценивать с помощью кросс-валидации по метрике f1. Предварительно сбалансируем классы

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=12345, class_weight='balanced')

In [17]:
final_score = cross_val_score(model, tf_idf_train, target_train, cv=3, scoring='f1')
final_score = final_score.sum() / len(final_score)
print('Средняя оценка качества модели:', final_score)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Средняя оценка качества модели: 0.7428330163523045


In [20]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import numpy as np
state=12345

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

Протестируем Древо решений

In [22]:
f1_best = 0
#estim_best = 0
depth_best = 0

for depth in range(2, 11):
    model = DecisionTreeClassifier(random_state = state, max_depth = depth, class_weight='balanced')
    f1_score = cross_val_score(model, tf_idf_train, target_train, cv=3, scoring='f1')
    f1_score = f1_score.sum() / len(f1_score)
    print('F1 score: {:.2f}'.format(f1_score))
    if f1_score>f1_best:
        f1_best=f1_score
        #estim_best=estim
        depth_best=depth

print('')
print('Наибольший F1 score = {:.2f} при значении n_estimators = {} и max_depth = {}'.format(f1_best, 3, depth_best))

F1 score: 0.32
F1 score: 0.37
F1 score: 0.43
F1 score: 0.47
F1 score: 0.50
F1 score: 0.50
F1 score: 0.53
F1 score: 0.54
F1 score: 0.37

Наибольший F1 score = 0.54 при значении n_estimators = 3 и max_depth = 9


Протестируем случайный лес при наилучшей глубине для решающего дерева (9), при количестве деревьев в диапазоне от 1 до 10 

In [23]:
f1_best = 0
estim_best = 0
#depth_best = 0

for estim in range(1, 11):
    model = RandomForestClassifier(random_state = state, n_estimators = estim, max_depth = 9, class_weight='balanced')
    f1_score = cross_val_score(model, tf_idf_train, target_train, cv=3, scoring='f1')
    f1_score = f1_score.sum() / len(f1_score)
    print('F1 score: {:.2f}'.format(f1_score))
    if f1_score>f1_best:
        f1_best=f1_score
        estim_best=estim
        #depth_best=depth

print('')
print('Наибольший F1 score = {:.2f} при значении n_estimators = {} и max_depth = {}'.format(f1_best, estim_best, 9))

F1 score: 0.20
F1 score: 0.21
F1 score: 0.22
F1 score: 0.23
F1 score: 0.24
F1 score: 0.25
F1 score: 0.25
F1 score: 0.26
F1 score: 0.27
F1 score: 0.27

Наибольший F1 score = 0.27 при значении n_estimators = 10 и max_depth = 9


Протестируем лучшие модели на тестовой выборке. Начнем с логичтической регресси

In [24]:
%%time
from sklearn.metrics import f1_score
model = LogisticRegression(random_state=12345, class_weight='balanced')
model.fit(tf_idf_train, target_train)
prediction = model.predict(tf_idf_test)
print('F1 score: {:.4f}'.format(f1_score(target_test, prediction)))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


F1 score: 0.7503
CPU times: user 8.66 s, sys: 5.35 s, total: 14 s
Wall time: 14 s


Решающее дерево

In [25]:
%%time
model = DecisionTreeClassifier(random_state = state, max_depth = 9, class_weight='balanced')
model.fit(tf_idf_train, target_train)
prediction = model.predict(tf_idf_test)
print('F1 score: {:.4f}'.format(f1_score(target_test, prediction)))

F1 score: 0.5325
CPU times: user 8.33 s, sys: 8.36 ms, total: 8.34 s
Wall time: 8.36 s


Случайный лес

In [26]:
%%time
model = RandomForestClassifier(random_state = state, n_estimators = 10, max_depth = 9, class_weight='balanced')
model.fit(tf_idf_train, target_train)
prediction = model.predict(tf_idf_test)
print('F1 score: {:.4f}'.format(f1_score(target_test, prediction)))

F1 score: 0.2952
CPU times: user 798 ms, sys: 3.78 ms, total: 802 ms
Wall time: 812 ms


# 3. Выводы

Требуемое качество метрики достигнуто при использовании модели машинного обучения - логистическая регрессия, используем ее для решения поставленной бизнес-задачи.